In [9]:
import torch_geometric as tg
from utils.load_md17 import load_md17

from models.se3_equivariant_transformer import Se3EquivariantTransformer
import torch


In [10]:
# Define an Equivariance test that should work for an arbitrary module
# Then we can just test all the modules in turn 

dd = load_md17(dataset_name='aspirin CCSD', dataset_dir='./../real_datasets/MD17/', radius=2)
train_data = dd['train']

In [11]:
graph = train_data[0]
graph.distances = torch.clone(graph.edge_attr)
